In [1]:
import numpy as np
from pynq import Overlay
from pynq import MMIO




In [17]:
def float_to_hex(flo):
    # convert floating point number to fixed point number
    # how to use
    # ex: print(float_to_hex(0.7254902))
    # output: 0B
    mp = {'0000':'0',
          '0001':'1',
          '0010':'2',
          '0011':'3',
          '0100':'4',
          '0101':'5',
          '0110':'6',
          '0111':'7',
          '1000':'8',
          '1001':'9',
          '1010':'A',
          '1011':'B',
          '1100':'C',
          '1101':'D',
          '1110':'E',
          '1111':'F',
         }
    flo = flo -  flo%(2**-4)
    ans = ''
    for i in range(3,-5,-1):
        if flo >= 2**i:
            flo -= 2**i
            ans += '1'
        else:
            ans += '0'
        # print(i)
    return mp[ans[0:4]]+mp[ans[4:8]]

def fix_to_float(in_hex):
    # no possible for negative number conversion
    # this function is not for general case
    # output after lru must be positive
    # expected length of input is 2 
    mp = {'0':'0000',
          '1':'0001',
          '2':'0010',
          '3':'0011',
          '4':'0100',
          '5':'0101',
          '6':'0110',
          '7':'0111',
          '8':'1000',
          '9':'1001',
          'A':'1010',
          'B':'1011',
          'C':'1100',
          'D':'1101',
          'E':'1110',
          'F':'1111'}
    
    binary_fmt = mp[in_hex[0]] + mp[in_hex[1]]
    ans = 0
    
    for i in range(7,-1,-1):
        #print(2**(i-4))
        ans += 2**(i-4) if binary_fmt[7-i] == '1' else 0
        #print(ans)
        
    return ans
    #print(binary_fmt)

def pass_dk1():
    dk1 = np.empty((0,128),float)
    row = np.array([])
    dk1f = open('dk1.txt','r')
    line = dk1f.readline()
    while line:
        for i in range(128): 
            row = np.append(row,float(line))
            line = dk1f.readline()
        dk1 = np.append(dk1,np.expand_dims(row, axis=0),axis=0)
        row = np.array([])
    dk1f.close()
    return dk1

def pass_dk2():
    dk2 = np.empty((0,10),float)
    row = np.array([])
    dk2f = open('dk2.txt','r')
    line = dk2f.readline()
    while line:
        for i in range(10): 
            row = np.append(row,float(line))
            line = dk2f.readline()
        dk2 = np.append(dk2,np.expand_dims(row, axis=0),axis=0)
        row = np.array([])
    dk2f.close()
    return dk2

def pass_db1():
    db1 = np.array([])
    with open('db1.txt') as f:
        for line in f:
            db1 = np.append(db1,float(line))
            if 'str' in line:
                  break
    return db1

def pass_db2():
    db2 = np.array([])
    with open('db2.txt') as f:
        for line in f:
            db2 = np.append(db2,float(line))
            if 'str' in line:
                  break
    return db2


def get_float_out1():
    # try to read data generate by hardware
    dk1 = np.zeros(676)
    index = 0
    dk1f = open('output1.hex','r')
    line = dk1f.readline()
    while line:
        for i in range(169): 
            #print(line[0:2])
            #print(line[2:4])
            #print(line[4:6])
            #print(line[6:8])
            dk1[index] = fix_to_float(line[0:2])
            dk1[index+1] = fix_to_float(line[2:4])
            dk1[index+2] = fix_to_float(line[4:6])
            dk1[index+3] = fix_to_float(line[6:8])
            index += 4
            line = dk1f.readline()
    return dk1

def get_float_out2():
    # try to read data generate by hardware
    dk2 = np.zeros(676)
    index = 0
    dk2f = open('output2.hex','r')
    line = dk2f.readline()
    while line:
        for i in range(169): 
            #print(line[0:2])
            #print(line[2:4])
            #print(line[4:6])
            #print(line[6:8])
            dk2[index] = fix_to_float(line[0:2])
            dk2[index+1] = fix_to_float(line[2:4])
            dk2[index+2] = fix_to_float(line[4:6])
            dk2[index+3] = fix_to_float(line[6:8])
            index += 4
            line = dk2f.readline()
    return dk2



def pass_x(path):
    x = np.empty((0,28),float)
    row = np.array([])
    xf = open(path,'r')
    line = xf.readline()
    while line:
        for i in range(28): 
            row = np.append(row,float(line))
            line = xf.readline()
        x = np.append(x,np.expand_dims(row, axis=0),axis=0)
        row = np.array([])
    xf.close()
    return x

def pass_y(path):
    y = np.array([])
    with open(path,'r') as f:
        for line in f:
            y = np.append(y,float(line))
            if 'str' in line:
                  break
    return y

In [37]:
'''

This block is just for one time operation of MNIST


'''


design = Overlay('./mnist_cnn.bit')

 
cdma_address = design.ip_dict['axi_cdma_0']['phys_addr']
axi_gpio_address = design.ip_dict['axi_gpio_0']['phys_addr']
 

zynq_addr = 0x30000000
cdma_ctrl = cdma_address+0x00
cdma_sa = cdma_address+0x18
cdma_da = cdma_address+0x20
cdma_btt = cdma_address+0x28

bram0_addr = 0xC0000000
bram1_addr = 0xC2000000
bram2_addr = 0xC4000000


zynq_sys = MMIO(zynq_addr,0x1500)
zynq_ans1 = MMIO(zynq_addr,0x4)
zynq_ans2 = MMIO(zynq_addr+4,0x4)
ctrl = MMIO(cdma_ctrl,0x18)
sa = MMIO(cdma_sa,0x1500)
da = MMIO(cdma_da,0x1500)
btt = MMIO(cdma_btt,0x10)

 
'''

initial input file to bram0
    
    
'''
#print('--- Start Initial! ---\n')

bram0_offset = 0x0

with open("fix_input_0.txt", "r") as f_in:
    for line1 in f_in:
        if not line1:
            break
        zynq_sys.write(bram0_offset,int('0x'+line1,16))      
        bram0_offset += 4
 

'''


 write '1' into AXI_GPIO for start!
 
 
'''      
count=0


ctrl.write(0x0,0x04)
sa.write(0x0,zynq_addr)
da.write(0x0,bram0_addr)
btt.write(0x0,0x1500)


#print('--- Initial Finish! ---\n')
gpio_a = MMIO(axi_gpio_address+8,8)
gpio_a.write(0x4,0)
gpio_a.write(0,1)

while count<100000:
    count+=1
    
gpio_a.write(0x4,0)
gpio_a.write(0,0)


 

 
'''
 use 'a' to append
 use 'w' to truncate

'''

bram1_offset = 0x0

ctrl.write(0x0,0x04)
sa.write(0x0,bram1_addr)
da.write(0x0,zynq_addr)
btt.write(0x0,0x1500)

dk1 = np.zeros(676)
index = 0
while bram1_offset < 169*4:
    line = str(hex(zynq_sys.read(bram1_offset))).upper()[2:]
    if len(line)!=8:
        line = '0'*(8-len(line)) + line
    bram1_offset += 4
    
    #for i in range(169):
    dk1[index] = fix_to_float(line[0:2])
    dk1[index+1] = fix_to_float(line[2:4])
    dk1[index+2] = fix_to_float(line[4:6])
    dk1[index+3] = fix_to_float(line[6:8])
    index += 4 
        
        
bram2_offset = 0x0

ctrl.write(0x0,0x04)
sa.write(0x0,bram2_addr)
da.write(0x0,zynq_addr)
btt.write(0x0,0x1500)
 
dk2 = np.zeros(676)
index = 0
while bram2_offset < 169*4:
    line = str(hex(zynq_sys.read(bram2_offset))).upper()[2:]
    if len(line)!=8:
        line = '0'*(8-len(line)) + line
    bram2_offset += 4
    
    #for i in range(169):
    dk2[index] = fix_to_float(line[0:2])
    dk2[index+1] = fix_to_float(line[2:4])
    dk2[index+2] = fix_to_float(line[4:6])
    dk2[index+3] = fix_to_float(line[6:8])
    index += 4 


        
#print('--- Hardware Convolution Finish! ---\n')
#print('--- Software Fully Connective Start ---\n')
r1 = np.array([])
r2 = np.array([])
l1_out1 = np.empty((0,26),float)
l1_out2 = np.empty((0,26),float)
d1_index = 0
d2_index = 0


# after layer 1 convolution ( get result from block memory)
# result store in dk1, dk2
# double `for loop` for transform 1D array into 2D array which named l1_out1 and l2_out2
for i in range(26):
    for j in range(26):
        r1 = np.append(r1,dk1[d1_index])
        d1_index += 1
        r2 = np.append(r2,dk2[d2_index])
        d2_index += 1
    l1_out1 = np.append(l1_out1,np.expand_dims(r1, axis=0),axis=0)
    l1_out2 = np.append(l1_out2,np.expand_dims(r2, axis=0),axis=0)
    r1 = np.array([])
    r2 = np.array([])
    
    
# layer 2 max pooling
maxp1 = -1
maxp2 = -1
l2_out1 = np.empty((0,13),float)
l2_out2 = np.empty((0,13),float)
for i in range(0,25,2):
    for j in range(0,25,2):
        for m in range(2):
            for n in range(2):
                maxp1 = max(maxp1,l1_out1[i+m,j+n]) 
                maxp2 = max(maxp2,l1_out2[i+m,j+n])
        r1 = np.append(r1,maxp1)
        r2 = np.append(r2,maxp2)
        maxp1 = -1
        maxp2 = -1
    l2_out1 = np.append(l2_out1,np.expand_dims(r1, axis=0),axis=0)
    l2_out2 = np.append(l2_out2,np.expand_dims(r2, axis=0),axis=0)  
    r1 = np.array([])
    r2 = np.array([])

# layer 3 flatten
l3_out = np.array([])

for i in range(13):
    for j in range(13):
        l3_out = np.append(l3_out,l2_out1[i,j])
        l3_out = np.append(l3_out,l2_out2[i,j])

dk1 = pass_dk1()
dk2 = pass_dk2()
db1 = pass_db1()
db2 = pass_db2()
# layer 4 dense

l4_out = np.dot(l3_out,dk1)
for i in range(128):
    l4_out[i] = max(l4_out[i]+db1[i],0)


# layer 5 dense with softmax

l5_out = np.dot(l4_out,dk2)
for i in range(10):
    l5_out[i] = l5_out[i]+db2[i]


# print('--- Software Operation Finish ---\n\n')
ans = np.exp(l5_out)/sum(np.exp(l5_out))
#for i in ans:
    #print("%.5f"%(i))

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


--- Start Initial! ---

0000050B

09090302

00000D0F

0F0F0F0F

0C0C0C0C

0C0C0C0C

0A030000

00000407

04070A0E

0F0E0F0F

0F0F0E0F

0F080000

00000001

04000404

0403010E

0F060000

0000050F

0D010000

00010E10

05000000

00080F0E

02000000

030F0F03

080F0B00

0C0F0300

00000007

0F0B0000

0000040F

0F030000

00010D0F

0A000000

000C0F0D

02000000

020F0F04

00000001

0E0F0700

00000008

0F0F0300

0000030F

0F0F0300

0000070F

0F0D0200

0000070F

0C010000

--- Initial Finish! ---

--- Hardware Convolution Finish! ---

--- Software Fully Connective Start ---

--- Software Operation Finish ---


0.10740
0.00000
0.00001
0.00006
0.00000
0.00702
0.00000
0.88497
0.00001
0.00051


In [52]:
'''

This block is definition of MNIST operation which is for the usage of multiple time MNIST operations

'''


def MNIST_OPERATION(input_path):

    ctrl = MMIO(cdma_ctrl,0x18)
    sa = MMIO(cdma_sa,0x1500)
    da = MMIO(cdma_da,0x1500)
    btt = MMIO(cdma_btt,0x10)


    '''

    initial input file to bram0


    '''
    #print('--- Start Initial! ---\n')

    bram0_offset = 0x0
    read_index = 0
    four_line = ''
    with open(input_path, "r") as f_in:
        for line1 in f_in:
            if not line1:
                break
            
             
            if read_index == 3:
                read_index = 0
                four_line += float_to_hex(float(line1))
                zynq_sys.write(bram0_offset,int('0x'+four_line,16))
                four_line = ''
                bram0_offset += 4
            else:
                read_index += 1
                four_line += float_to_hex(float(line1))
            
            
                
            


    '''


     write '1' into AXI_GPIO for start!


    '''      
    count=0


    ctrl.write(0x0,0x04)
    sa.write(0x0,zynq_addr)
    da.write(0x0,bram0_addr)
    btt.write(0x0,0x1500)


    #print('--- Initial Finish! ---\n')
    gpio_a = MMIO(axi_gpio_address+8,8)
    gpio_a.write(0x4,0)
    gpio_a.write(0,1)

    while count<100000:
        count+=1

    gpio_a.write(0x4,0)
    gpio_a.write(0,0)





    '''
     use 'a' to append
     use 'w' to truncate

    '''

    bram1_offset = 0x0

    ctrl.write(0x0,0x04)
    sa.write(0x0,bram1_addr)
    da.write(0x0,zynq_addr)
    btt.write(0x0,0x1500)
 
    dk1 = np.zeros(676)
    index = 0
    while bram1_offset < 169*4:
        line = str(hex(zynq_sys.read(bram1_offset))).upper()[2:]
        if len(line)!=8:
            line = '0'*(8-len(line)) + line
        bram1_offset += 4

        #for i in range(169):
        dk1[index] = fix_to_float(line[0:2])
        dk1[index+1] = fix_to_float(line[2:4])
        dk1[index+2] = fix_to_float(line[4:6])
        dk1[index+3] = fix_to_float(line[6:8])
        index += 4 


    bram2_offset = 0x0

    ctrl.write(0x0,0x04)
    sa.write(0x0,bram2_addr)
    da.write(0x0,zynq_addr)
    btt.write(0x0,0x1500)
 
    dk2 = np.zeros(676)
    index = 0
    while bram2_offset < 169*4:
        line = str(hex(zynq_sys.read(bram2_offset))).upper()[2:]
        if len(line)!=8:
            line = '0'*(8-len(line)) + line
        bram2_offset += 4

        #for i in range(169):
        dk2[index] = fix_to_float(line[0:2])
        dk2[index+1] = fix_to_float(line[2:4])
        dk2[index+2] = fix_to_float(line[4:6])
        dk2[index+3] = fix_to_float(line[6:8])
        index += 4 



    #print('--- Hardware Convolution Finish! ---\n')
    #print('--- Software Fully Connective Start ---\n')
    r1 = np.array([])
    r2 = np.array([])
    l1_out1 = np.empty((0,26),float)
    l1_out2 = np.empty((0,26),float)
    d1_index = 0
    d2_index = 0


    # after layer 1 convolution ( get result from block memory)
    # result store in dk1, dk2
    # double `for loop` for transform 1D array into 2D array which named l1_out1 and l2_out2
    for i in range(26):
        for j in range(26):
            r1 = np.append(r1,dk1[d1_index])
            d1_index += 1
            r2 = np.append(r2,dk2[d2_index])
            d2_index += 1
        l1_out1 = np.append(l1_out1,np.expand_dims(r1, axis=0),axis=0)
        l1_out2 = np.append(l1_out2,np.expand_dims(r2, axis=0),axis=0)
        r1 = np.array([])
        r2 = np.array([])


    # layer 2 max pooling
    maxp1 = -1
    maxp2 = -1
    l2_out1 = np.empty((0,13),float)
    l2_out2 = np.empty((0,13),float)
    for i in range(0,25,2):
        for j in range(0,25,2):
            for m in range(2):
                for n in range(2):
                    maxp1 = max(maxp1,l1_out1[i+m,j+n]) 
                    maxp2 = max(maxp2,l1_out2[i+m,j+n])
            r1 = np.append(r1,maxp1)
            r2 = np.append(r2,maxp2)
            maxp1 = -1
            maxp2 = -1
        l2_out1 = np.append(l2_out1,np.expand_dims(r1, axis=0),axis=0)
        l2_out2 = np.append(l2_out2,np.expand_dims(r2, axis=0),axis=0)  
        r1 = np.array([])
        r2 = np.array([])

    # layer 3 flatten
    l3_out = np.array([])

    for i in range(13):
        for j in range(13):
            l3_out = np.append(l3_out,l2_out1[i,j])
            l3_out = np.append(l3_out,l2_out2[i,j])

    dk1 = pass_dk1()
    dk2 = pass_dk2()
    db1 = pass_db1()
    db2 = pass_db2()
    # layer 4 dense

    l4_out = np.dot(l3_out,dk1)
    for i in range(128):
        l4_out[i] = max(l4_out[i]+db1[i],0)


    # layer 5 dense with softmax

    l5_out = np.dot(l4_out,dk2)
    for i in range(10):
        l5_out[i] = l5_out[i]+db2[i]


    #print('--- Software Operation Finish ---\n\n')
    ans = np.exp(l5_out)/sum(np.exp(l5_out))
    guess_number = 0
    proba = -1
    for i,j in zip(ans,range(10)):
        print("Probability of ",j,end = '')
        print(" is : %.5f"%(i))
        #print(j)
        proba = i if i > proba else proba
        guess_number = j if i == proba else guess_number
        
    #print(guess_number)
    return guess_number






    '''    
    v = 0
    ma = -1
    for i in range(10):
        if ans[i]>ma:
            ma = ans[i]
            v = i

    if y[v]>0.5:
        return True
    else: 
        return False
    '''


In [51]:
'''

Example for using `MNIST_OPERATION`

'''


# initial shared variable (start)


design = Overlay('./mnist_cnn.bit')

cdma_address = design.ip_dict['axi_cdma_0']['phys_addr']
axi_gpio_address = design.ip_dict['axi_gpio_0']['phys_addr']


zynq_addr = 0x30000000
cdma_ctrl = cdma_address+0x00
cdma_sa = cdma_address+0x18
cdma_da = cdma_address+0x20
cdma_btt = cdma_address+0x28

bram0_addr = 0xC0000000
bram1_addr = 0xC2000000
bram2_addr = 0xC4000000


zynq_sys = MMIO(zynq_addr,0x1500)
zynq_ans1 = MMIO(zynq_addr,0x4)
zynq_ans2 = MMIO(zynq_addr+4,0x4)



# initial shared variable (finish)




# Example for how to use MNIST_OPERATION

# print(MNIST_OPERATION('../test_data_x/test_data_x_1.txt'))

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


0.00000
0.00025
0.99975
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
0.00000
2
